# 기본환경 설정

In [1]:
from google.colab import userdata
import huggingface_hub

In [2]:
HF_KEY = userdata.get("HF_KEY")
huggingface_hub.login(HF_KEY)

# 데이터 준비

In [11]:
import os
import sentencepiece as spm
from datasets import load_dataset

In [12]:
os.makedirs("spm_models", exist_ok=True)

## 영어

In [4]:
dataset_eng = load_dataset("ag_news", split="train[:100]")

README.md: 0.00B [00:00, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/18.6M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

In [5]:
sentences_eng = dataset_eng["text"]
sentences_eng[:2]

["Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.",
 'Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group,\\which has a reputation for making well-timed and occasionally\\controversial plays in the defense industry, has quietly placed\\its bets on another part of the market.']

In [6]:
with open("corpus_eng.txt", "w", encoding="utf-8") as f:
    for line in sentences_eng:
        f.write(line + "\n")

## 한국어

In [7]:
dataset_kor = load_dataset("daekeun-ml/naver-news-summarization-ko", split="train[:100]")

README.md:   0%|          | 0.00/787 [00:00<?, ?B/s]

train.csv:   0%|          | 0.00/66.3M [00:00<?, ?B/s]

validation.csv: 0.00B [00:00, ?B/s]

test.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/22194 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2466 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2740 [00:00<?, ? examples/s]

In [8]:
sentences_kor = dataset_kor["document"]
sentences_kor[:2]

['앵커 정부가 올해 하반기 우리 경제의 버팀목인 수출 확대를 위해 총력을 기울이기로 했습니다. 특히 수출 중소기업의 물류난 해소를 위해 무역금융 규모를 40조 원 이상 확대하고 물류비 지원과 임시선박 투입 등을 추진하기로 했습니다. 류환홍 기자가 보도합니다. 기자 수출은 최고의 실적을 보였지만 수입액이 급증하면서 올해 상반기 우리나라 무역수지는 역대 최악인 103억 달러 적자를 기록했습니다. 정부가 수출확대에 총력을 기울이기로 한 것은 원자재 가격 상승 등 대외 리스크가 가중되는 상황에서 수출 증가세 지속이야말로 한국경제의 회복을 위한 열쇠라고 본 것입니다. 추경호 경제부총리 겸 기획재정부 장관 정부는 우리 경제의 성장엔진인 수출이 높은 증가세를 지속할 수 있도록 총력을 다하겠습니다. 우선 물류 부담 증가 원자재 가격 상승 등 가중되고 있는 대외 리스크에 대해 적극 대응하겠습니다. 특히 중소기업과 중견기업 수출 지원을 위해 무역금융 규모를 연초 목표보다 40조 원 늘린 301조 원까지 확대하고 물류비 부담을 줄이기 위한 대책도 마련했습니다. 이창양 산업통상자원부 장관 국제 해상운임이 안정될 때까지 월 4척 이상의 임시선박을 지속 투입하는 한편 중소기업 전용 선복 적재 용량 도 현재보다 주당 50TEU 늘려 공급하겠습니다. 하반기에 우리 기업들의 수출 기회를 늘리기 위해 2 500여 개 수출기업을 대상으로 해외 전시회 참가를 지원하는 등 마케팅 지원도 벌이기로 했습니다. 정부는 또 이달 중으로 반도체를 비롯한 첨단 산업 육성 전략을 마련해 수출 증가세를 뒷받침하고 에너지 소비를 줄이기 위한 효율화 방안을 마련해 무역수지 개선에 나서기로 했습니다. YTN 류환홍입니다.',
 '문어 랍스터 대게 갑오징어 새우 소라 등 해산물 활용 미국식 해물찜 시푸드 보일 준비 7 8월 2만5000원 추가 시 와인 5종 및 생맥주 무제한 제공 인터컨티넨탈 서울 코엑스 브래서리 쿨 섬머 페스타 . 인터컨티넨탈 서울 코엑스 1층 뷔페 레스토랑 브래서리는 오는 6일부터 8월31일까지 

In [9]:
with open("corpus_kor.txt", "w", encoding="utf-8") as f:
    for line in sentences_kor:
        f.write(line + "\n")

# SentencePiece 모델 학습

## 영어

In [13]:
spm.SentencePieceTrainer.train(
    input="corpus_eng.txt",
    model_prefix="spm_models/spm_eng",
    vocab_size=1000,
    model_type="bpe" # bpe, unigram, char 등 가능
)

## 한국어

In [14]:
spm.SentencePieceTrainer.train(
    input="corpus_kor.txt",
    model_prefix="spm_models/spm_kor",
    vocab_size=1000,
    model_type="bpe" # bpe, unigram, char 등 가능
)

# SentencePiece 모델 로드 & 토큰화

In [15]:
tokenizer_eng = spm.SentencePieceProcessor(model_file="spm_models/spm_eng.model")
tokenizer_kor = spm.SentencePieceProcessor(model_file="spm_models/spm_kor.model")

In [16]:
# 예제 문장
example_eng = "I love natural language processing."
example_kor = "저는 자연어 처리를 공부하고 있습니다."

In [17]:
tokens_eng = tokenizer_eng.encode(example_eng, out_type=str)
tokens_kor = tokenizer_kor.encode(example_kor, out_type=str)

In [18]:
print(f"영어 토큰: {tokens_eng}\n한글 토큰: {tokens_kor}")

영어 토큰: ['▁I', '▁lo', 've', '▁n', 'atur', 'al', '▁l', 'ang', 'u', 'age', '▁pro', 'cess', 'ing', '.']
한글 토큰: ['▁', '저', '는', '▁자', '연', '어', '▁', '처', '리', '를', '▁공', '부', '하고', '▁있', '습', '니다', '.']


# 인덱스 변환 및 임베딩 : PyTorch 예시

In [19]:
import torch
from torch import nn

In [21]:
# 문자를 토큰 인덱스로 변환
ids_eng = torch.LongTensor([tokenizer_eng.encode(example_eng)])
ids_kor = torch.LongTensor([tokenizer_kor.encode(example_kor)])

In [22]:
# 임베딩 설정
emb_dim = 16
embedding_eng = nn.Embedding(tokenizer_eng.get_piece_size(), emb_dim)
embedding_kor = nn.Embedding(tokenizer_kor.get_piece_size(), emb_dim)

In [23]:
embedding_eng, embedding_kor

(Embedding(1000, 16), Embedding(1000, 16))

In [24]:
# 임베딩 벡터 생성
embeds_eng = embedding_eng(ids_eng)
embeds_kor = embedding_eng(ids_kor)

In [25]:
embeds_eng.shape, embeds_kor.shape

(torch.Size([1, 14, 16]), torch.Size([1, 17, 16]))

In [31]:
ids_eng

tensor([[ 56, 229,  75,  71, 821,  25,  68, 407, 940, 351, 127, 631,  16, 948]])

In [27]:
embeds_eng

tensor([[[-5.7892e-01,  8.3517e-01,  1.5578e+00, -5.1714e-02,  6.3957e-01,
          -7.9588e-01, -4.2883e-02, -1.3438e+00, -6.5932e-01,  6.3524e-01,
          -1.2586e+00,  1.6119e+00,  3.7857e-01, -1.5591e+00, -2.6279e-01,
           8.3302e-01],
         [-2.2292e-01, -1.4203e+00, -1.5657e+00,  1.4855e+00,  1.0192e+00,
          -1.5360e+00, -2.1583e+00, -1.9677e-01, -8.6825e-02, -7.9061e-01,
           8.8039e-02,  6.7380e-01, -7.9969e-01, -5.4504e-01, -4.6406e-01,
           1.7249e+00],
         [-1.2349e+00,  7.1495e-01,  2.0093e-01, -1.9907e+00, -4.8439e-01,
           6.9951e-01,  6.4923e-01,  1.2819e+00,  8.4474e-01, -5.6583e-01,
           1.3569e+00, -4.7218e-01, -9.1351e-02,  9.5515e-01, -2.5093e-01,
           3.7132e-01],
         [ 3.4171e-01,  8.2669e-01,  1.1774e+00,  7.9792e-01, -1.6563e+00,
           1.5320e+00,  2.9402e-01, -8.9833e-01, -4.7052e-01,  1.7625e+00,
           1.0051e+00,  4.1430e-02,  1.0848e+00,  9.9405e-01, -2.1705e-01,
           4.3545e-02],
    

# 토큰화 방법 비교

In [32]:
# 1) 코퍼스 파일 만들기
corpus = """I love natural language processing.
I love machine learning.
ChatGPT is amazing.
Natural language models are powerful."""
with open("corpus.txt", "w", encoding="utf-8") as f:
    f.write(corpus)

In [37]:
# 2) unigram & bpe 모델 학습 (vocab_size=30, character_coverage=1.0)
for mtype in ["unigram", "bpe"]:
    spm.SentencePieceTrainer.train(
        input="corpus.txt",
        model_prefix=f"spm_models/spm_{mtype}",
        vocab_size=40,
        model_type=mtype,
        character_coverage=1.0
    )

In [38]:
# 3) 학습된 모델 로드
sp_unigram = spm.SentencePieceProcessor(model_file="spm_models/spm_unigram.model")
sp_bpe     = spm.SentencePieceProcessor(model_file="spm_models/spm_bpe.model")

In [39]:
# 4) 샘플 문장 토큰화
sample = "I love natural language processing."
tokens_unigram = sp_unigram.encode(sample, out_type=str)
tokens_bpe     = sp_bpe.encode(sample,     out_type=str)

In [40]:
# 5) char & word 방식은 수동 분할
tokens_char = [c if c != " " else "▁" for c in sample]
tokens_word = sample.split(" ")

In [41]:
# 6) 결과 출력
print("unigram:", tokens_unigram, "→", len(tokens_unigram), "tokens")
print("bpe    :", tokens_bpe,     "→", len(tokens_bpe),     "tokens")
print("char   :", tokens_char,    "→", len(tokens_char),    "tokens")
print("word   :", tokens_word,    "→", len(tokens_word),    "tokens")

unigram: ['▁', 'I', '▁l', 'o', 'v', 'e', '▁', 'n', 'at', 'u', 'r', 'a', 'l', '▁l', 'a', 'ng', 'u', 'a', 'g', 'e', '▁p', 'r', 'o', 'c', 'e', 's', 's', 'in', 'g', '.'] → 30 tokens
bpe    : ['▁', 'I', '▁l', 'ov', 'e', '▁', 'n', 'at', 'u', 'r', 'al', '▁l', 'a', 'ng', 'u', 'ag', 'e', '▁', 'p', 'r', 'o', 'c', 'e', 's', 's', 'ing', '.'] → 27 tokens
char   : ['I', '▁', 'l', 'o', 'v', 'e', '▁', 'n', 'a', 't', 'u', 'r', 'a', 'l', '▁', 'l', 'a', 'n', 'g', 'u', 'a', 'g', 'e', '▁', 'p', 'r', 'o', 'c', 'e', 's', 's', 'i', 'n', 'g', '.'] → 35 tokens
word   : ['I', 'love', 'natural', 'language', 'processing.'] → 5 tokens
